In [1]:
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWh_odlDhvB_s1ATYr9du6aQWTW4BQ4LPQo6pcIRrcJhx-fo-CKshjI



Successfully saved authorization token.


In [2]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles = map_id_dict['tile_fetcher'].url_format,
        attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name = name,
        overlay = True,
        control = True
    ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [3]:
# https://developers.google.com/earth-engine/datasets/catalog/ECMWF_ERA5_DAILY
era5_2mt = (
    ee.ImageCollection('ECMWF/ERA5/DAILY')
    .select('mean_2m_air_temperature')
    .filter(ee.Filter.date('2019-07-01', '2019-07-31'))
)

vis2mt = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    era5_2mt.reduce(ee.Reducer.mean()),
    vis2mt,
    '2 meter temperature'
)
display(my_map)

In [4]:
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET
idaho_data = (
    ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
    .select('tmmn')
    .filter(ee.Filter.date('2019-07-01', '2020-07-01'))
)

vis = {
  'min': 250,
  'max': 320,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    idaho_data.reduce(ee.Reducer.mean()),
    vis,
    'idaho avg temp'
)

display(my_map)

In [5]:
# https://developers.google.com/earth-engine/datasets/catalog/IDAHO_EPSCOR_GRIDMET
idaho_data = (
    ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
    .select('pr')
    .filter(ee.Filter.date('2019-07-01', '2020-07-01'))
)

vis = {
  'min': 0,
  'max': 2000,
  'palette': [
    '#000080', '#0000D9', '#4000FF', '#8000FF', '#0080FF', '#00FFFF', '#00FF80',
    '#80FF00', '#DAFF00', '#FFFF00', '#FFF500', '#FFDA00', '#FFB000', '#FFA400',
    '#FF4F00', '#FF2500', '#FF0A00', '#FF00FF'
  ]
}

my_map = folium.Map()
my_map.add_ee_layer(
    idaho_data.reduce(ee.Reducer.sum()),
    vis,
    'idaho total precip'
)

display(my_map)

In [6]:
precipitation = (
    ee.ImageCollection("IDAHO_EPSCOR/GRIDMET")
    .select('pr')
    .filter(ee.Filter.date('2019-07-01', '2020-07-01'))
).reduce(ee.Reducer.sum())

In [7]:
precipitation

In [8]:
precipitation.getInfo()

{'type': 'Image',
 'bands': [{'id': 'pr_sum',
   'data_type': {'type': 'PixelType', 'precision': 'double'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}]}

In [17]:
lon, lat = -72.26, 42.41
point = ee.Geometry.Point(lon, lat)
collection = ee.ImageCollection("IDAHO_EPSCOR/GRIDMET").filter(ee.Filter.date('2019-07-01', '2020-07-01'))
data = collection.getRegion(point, 1000).getInfo()

In [21]:
import pandas as pd
columns = data[0]
rows = []
for record in data[1:]:
    rows.append({
        c: v
        for c, v in zip(columns, record)
    })
df = pd.DataFrame(rows)

In [23]:
df.sum()

id           2019070120190702201907032019070420190705201907...
longitude                                             -26445.7
latitude                                               15523.5
time                                           577448740800000
pr                                                      1121.8
rmax                                                   31174.4
rmin                                                   15871.5
sph                                                    2.12104
srad                                                   65133.8
th                                                       80360
tmmn                                                    101292
tmmx                                                    105593
vs                                                      1520.6
erc                                                       8859
eto                                                     1059.6
bi                                                     

In [29]:
collection.select('pr').reduce(ee.Reducer.sum()).sample(point, 1000).getInfo()

{'type': 'FeatureCollection',
 'columns': {'pr_sum': 'Float'},
 'properties': {'band_order': ['pr_sum']},
 'features': [{'type': 'Feature',
   'geometry': None,
   'id': '0',
   'properties': {'pr_sum': 1121.800003260374}}]}